Disclaimer: This workbook is optional content.

The purpose of this notebook is to create repeatable code using classes. When creating the two moving boxes code, did you see how large the code looked and looked like it repeated itself? Did you wonder if that code could be shortened?

Classes would allow for that, but since they are beyond the scope of this SCIP summer session, so for those who are ready for more advanced programming concepts, this workbok is for you. First, I'll link you to a YouTube playlist that will teach you the basics of classes (called object-oriented programming, or OOP). Next, we'll use a class to recreate the two moving boxes code. And finally, I'll show you other advantages of using classes.


# Section 1: YouTube Playlist

Use the following link to become familiar with how classes work and how you use them:
https://www.youtube.com/watch?v=ZDa-Z5JzLYM&list=PL-osiE80TeTsqhIuOqKhwlXsIBIdSeYtc

This is a 6 video series by Corey Schafer and I strongly recommend doing the examples with him to get a sense of how to create and use classes. However, if you're already familiar with it, jump to Section 2.

One way to think about classes is they're like different careers; for example, the lawyer class studies the law, the scientist classes studies science, and the accountant class keeps track of the money.

# Section 2: MyBox Class

For this section, we're going to create the MyBox class to hold all the properties of the box, like where it will start and what color the box will be. Let's load up the libraries we will need for this notebook.

In [ ]:
# Import Libraries For this Notebook
import cv2
import numpy as np

from google.colab.patches import cv2_imshow
from IPython import display
from random import randint
from time import sleep

Now for the actual class. You will noticed I do break some class creation rules, I neglected to put in getters and setters for stepsize and some of the other variables.

In [ ]:
# Custom Box Class Code, for easier Box creation

class MyBox:
  def __init__(self, width, height, channels):
    # Setup dimensions of box and create it.
    self.width = width
    self.height = height
    self.channels = channels
    self.x = width
    self.y = height
    self.box = np.zeros((height, width, channels), dtype='uint8') # black box

    # Setup other constants
    # BGR Indices
    self.chanB = 0
    self.chanG = 1
    self.chanR = 2

    # Box Location (top-left)
    self.x0 = 0
    self.y0 = 0

    # Step Size
    self.stepSize = 20
    self.moveX = self.stepSize
    self.moveY = self.stepSize

    # Min/Max Color Values (For Random Color Creation)
    self.min = 0
    self.max = 255
  
  # Change colors of box
  def change_color(self, channel, value):
    self.box[:, :, channel] = value
  
  def make_blue(self):
    self.change_color(self.chanB, 255)
    self.change_color(self.chanG, 0)
    self.change_color(self.chanR, 0)
  
  def make_green(self):
    self.change_color(self.chanG, 255)
    self.change_color(self.chanB, 0)
    self.change_color(self.chanR, 0)

  def make_red(self):
    self.change_color(self.chanR, 255)
    self.change_color(self.chanG, 0)
    self.change_color(self.chanB, 0)
  
  # Randomize Box Color, use a for loop to go through all 3 channels.
  def randomize_color(self):
    for ch in range(self.channels):
      value = randint(self.min, self.max)
      self.change_color(ch, value)

  # Make box move left, for the second box
  def move_left(self):
    self.moveX = -self.stepSize
  
  # Incrementing box movement for each frame.
  def increment_box(self):
    self.x0+=self.moveX 
    self.y0+=self.moveY
  
  # The boundary check function uses the input resolution of the background image
  def boundary_check(self, xRez, yRez):
    if self.x0 < 0:
        self.moveX = self.stepSize # make the increment positive
        self.x0 = 0
    if self.y0 < 0:
        self.moveY = self.stepSize  # make the increment positive
        self.y0 = 0
    if self.x0 + self.x >= xRez:
        self.moveX = -self.stepSize  # make the increment negative
        self.x0 = xRez - self.x - 1
    if self.y0 + self.y >= yRez:
        self.moveY = -self.stepSize  # make the increment negative
        self.y0 = yRez - self.y - 1
  
  # Draws the actual box onto the background.
  # Optionally returns displayIM, but since Python is by reference, not necessary.
  # The displayIM will change
  def draw_box(self, displayIM):
    displayIM[self.y0:self.y0+self.y, self.x0:self.x0+self.x,:] = self.box  # paste sprite image into black display

    return displayIM
  
  # Manager method that will call the 3 functions to update and draw box.
  def update_and_draw_box(self, xRez, yRez, displayIM):
    self.increment_box()
    self.boundary_check(xRez, yRez)
    displayIM = self.draw_box(displayIM)
    return displayIM
  
print("Done Creating Class")

Now that the class is created, let's see how it works. First, let's recreate the two intersecting boxes like in the first Tracking Primer Notebook.

In [ ]:
# Bouncing Box Code using Classes

CYCLES=20 # number of times to move box

xRez=500; yRez=300; COLOR_CHANNELS=3  # black background dimensions

# Make Box 1
b1 = MyBox(100, 100, 3)
b1.make_red()

# Make Box 2, using function keywords, makes it easier to read
b2 = MyBox(width=100, height=10, channels=3)
b2.make_blue()

# Notice the math used to make sure box 2 has space on the right side
# I should put a setter here, but the Notebook was getting large already,
# so I opted to do this as a temporary measure
rMargin = 10
b2.x0 = xRez - (b2.x + rMargin)
# Make box move left instead of right
b2.move_left()

# For Loop using separate methods, to show how large the code could be
for i in range(CYCLES):    
    # increment moveX and moveY to move the box
    b1.increment_box()
    b1.boundary_check(xRez, yRez)

    b2.increment_box()
    b2.boundary_check(xRez, yRez)

    # move image
    displayIM=np.zeros((yRez,xRez,COLOR_CHANNELS),dtype='uint8') # black display

    # Python is by reference, so you don't need to have the displayIM assignment below
    displayIM = b1.draw_box(displayIM)
    displayIM = b2.draw_box(displayIM)

    # Order matters here, whatever box is drawn last will be "on top"
    
    cv2_imshow(displayIM)
    sleep(.5) # delay so we can see the image (in seconds)
    display.clear_output(wait=True)
   
print("Reached end of animation")

cv2.destroyAllWindows()

Cool, we recreated the two moving boxes from before. How about shrinking the code even further?

In [ ]:
# Assumes MyBox Class Code was run

CYCLES=20 # number of times to move box
xRez=500; yRez=300; COLOR_CHANNELS=3  # black background dimensions

b1 = MyBox(width=100, height=100, channels=COLOR_CHANNELS)
b1.make_red()

b2 = MyBox(width=100, height=10, channels=COLOR_CHANNELS)
b2.make_blue()

# Start box 2 opposite of box 1, making sure there is a gap on the right side,
# using rMargin
rMargin = 10
b2.x0 = xRez - (b2.x + rMargin)
b2.move_left() # Make box start moving left as default (to intersect with box1)

image_list = []

# For Loop using combined methods (looks smaller, but may confuse students)
for i in range(CYCLES):
    # move image
    displayIM=np.zeros((yRez,xRez,COLOR_CHANNELS),dtype='uint8') # black display
    
    # Increment box, check boundary, and draw box in displayIM
    displayIM = b1.update_and_draw_box(xRez, yRez, displayIM)
    displayIM = b2.update_and_draw_box(xRez, yRez, displayIM)

    # Order matters here, whatever box is drawn last will be "on top"
    # when crossing paths

    # Make a copy, very important!
    # Otherwise you end up with a movie of the same image
    frame = displayIM.copy()

    # Add frame to image_list
    image_list.append(frame)

    cv2_imshow(displayIM)
    sleep(.5) # delay so we can see the image (in seconds)
    display.clear_output(wait=True)

print("Reached end of animation")

cv2.destroyAllWindows()

# checking if amount of images saved matches number cycles.
# If they match, images saved should work.
print(f"image_list len: {len(image_list)}")
print(f"CYCLES: {CYCLES}")

Excellent, the code in the for loop shrunk even more, from a bunch of lines to only two lines for the updating and drawing of the box. Now that we recreated a known example. Let's go a little crazy. How about randomizing the box colors? 

Try running the code a few different times. Did you notice how the colors change each time?

In [ ]:
# Assumes MyBox Class Code was run

CYCLES=20 # number of times to move box
xRez=500; yRez=300; COLOR_CHANNELS=3  # black background dimensions

b1 = MyBox(width=100, height=100, channels=COLOR_CHANNELS)
b1.randomize_color()

b2 = MyBox(width=100, height=10, channels=COLOR_CHANNELS)
b2.randomize_color()

# Start box 2 opposite of box 1, making sure there is a gap on the right side,
# using rMargin
rMargin = 10
b2.x0 = xRez - (b2.x + rMargin)
b2.move_left() # Make box start moving left as default (to intersect with box1)

image_list = []

# For Loop using combined methods (looks smaller, but may confuse students)
for i in range(CYCLES):
    # move image
    displayIM=np.zeros((yRez,xRez,COLOR_CHANNELS),dtype='uint8') # black display
    
    # Increment box, check boundary, and draw box in displayIM
    displayIM = b1.update_and_draw_box(xRez, yRez, displayIM)
    displayIM = b2.update_and_draw_box(xRez, yRez, displayIM)

    # Order matters here, whatever box is drawn last will be "on top"
    # when crossing paths

    # Make a copy, very important!
    # Otherwise you end up with a movie of the same image
    frame = displayIM.copy()

    # Add frame to image_list
    image_list.append(frame)

    cv2_imshow(displayIM)
    sleep(.5) # delay so we can see the image (in seconds)
    display.clear_output(wait=True)

print("Reached end of animation")

cv2.destroyAllWindows()

# checking if amount of images saved matches number cycles.
# If they match, images saved should work.
print(f"image_list len: {len(image_list)}")
print(f"CYCLES: {CYCLES}")

That's nice and all, but how is this supposed to help? Two ways, to demonstrate how to make your code more readable and repeatable. And two, to truly test your algorithms, you sometimes need to create different scenarios to test those algorithms about; like with the object detection code.

So could we add more than two boxes and could they be detected. With classes, that will be easy!

In [ ]:
# Assumes MyBox Class Code was run

CYCLES=20 # number of times to move box
xRez=500; yRez=300; COLOR_CHANNELS=3  # black background dimensions

b1 = MyBox(width=100, height=100, channels=COLOR_CHANNELS)
b1.randomize_color()

b2 = MyBox(width=100, height=10, channels=COLOR_CHANNELS)
b2.randomize_color()

b3 = MyBox(width=50, height=30, channels=COLOR_CHANNELS)
b3.randomize_color()

# Start box 2 opposite of box 1, making sure there is a gap on the right side,
# using rMargin
rMargin = 10
b2.x0 = xRez - (b2.x + rMargin)
b2.move_left() # Make box start moving left as default (to intersect with box1)

# Make box 3 start in the middle
b3.x0 = int(xRez / 2)


image_list = []

# For Loop using combined methods (looks smaller, but may confuse students)
for i in range(CYCLES):
    # move image
    displayIM=np.zeros((yRez,xRez,COLOR_CHANNELS),dtype='uint8') # black display
    
    # Increment box, check boundary, and draw box in displayIM
    displayIM = b1.update_and_draw_box(xRez, yRez, displayIM)
    displayIM = b2.update_and_draw_box(xRez, yRez, displayIM)
    displayIM = b3.update_and_draw_box(xRez, yRez, displayIM)
    # Order matters here, whatever box is drawn last will be "on top"
    # when crossing paths

    # Make a copy, very important!
    # Otherwise you end up with a movie of the same image
    frame = displayIM.copy()

    # Add frame to image_list
    image_list.append(frame)

    cv2_imshow(displayIM)
    sleep(.5) # delay so we can see the image (in seconds)
    display.clear_output(wait=True)

print("Reached end of animation")

cv2.destroyAllWindows()

# checking if amount of images saved matches number cycles.
# If they match, images saved should work.
print(f"image_list len: {len(image_list)}")
print(f"CYCLES: {CYCLES}")

Look at that, just a few lines of code and now you have 3 different boxes bouncing around. I will leave it to you to test out if object detection works on these boxes.

The following sections are even more optional.

For even more advanced shenanigans, what if you wanted to really test your object detection code with all sorts of random elements like different widths and heights for any number of boxes? It might be easier to use a for loop create a list of objects (or boxes), and another for loop to update their location.

In [ ]:
# Assumes MyBox Class Code was run

CYCLES=40 # number of times to move box
xRez=500; yRez=300; COLOR_CHANNELS=3  # black background dimensions
NUM_BOX = 4  # Number of boxes you want to make

# Use for loop to create list of boxes
box_list = []

for i in range(NUM_BOX):
  # Make a random sized box within 10-100 width and height
  box = MyBox(width=randint(10, 100), height=randint(10, 100), channels=COLOR_CHANNELS)
  box.randomize_color()
  box_list.append(box)

image_list = []

# For Loop using combined methods (looks smaller, but may confuse students)
for i in range(CYCLES):
    # move image
    displayIM=np.zeros((yRez,xRez,COLOR_CHANNELS),dtype='uint8') # black display
    
    for box in box_list:
      box.update_and_draw_box(xRez, yRez, displayIM)

    # Make a copy, very important!
    # Otherwise you end up with a movie of the same image
    frame = displayIM.copy()

    # Add frame to image_list
    image_list.append(frame)

    cv2_imshow(displayIM)
    sleep(.5) # delay so we can see the image (in seconds)
    display.clear_output(wait=True)

print("Reached end of animation")

cv2.destroyAllWindows()

# checking if amount of images saved matches number cycles.
# If they match, images saved should work.
print(f"image_list len: {len(image_list)}")
print(f"CYCLES: {CYCLES}")

Because of the way the code is written, you'll have to increase the number of cycles to actually see all the boxes since they start at the same location.

A bonus task for you, could you randomize the location of the box, then use the boundary_check() method to make sure the box is within bounds? How creating a method that randomizes the width and height? The possibilities are endless!

In conclusion, you learned how to create a MyBox object, used it to recreate previous work, and then expanded upon it. In the science world, when you think of a new way to do research, you have to see how your new method compares with the old method. And that's exactly what we did here. Congratulations!